In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tarfile
import os
import sys
sys.path.append(os.path.abspath(".."))
from modules.data_import import loadData
print('ok')
general, employeeSurvey, managerSurvey, inTime, outTime = loadData()

In [ ]:
print("Old Shape: ", general.shape)
# IQR
# Calculate the upper and lower limits
Q1 = general['DistanceFromHome'].quantile(0.25)
Q3 = general['DistanceFromHome'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR

# Create arrays of Boolean values indicating the outlier rows
upper_array = np.where(general['DistanceFromHome'] >= upper)[0]
lower_array = np.where(general['DistanceFromHome'] <= lower)[0]

# Removing the outliers
general.drop(index=upper_array, inplace=True)
general.drop(index=lower_array, inplace=True)

# Print the new shape of the DataFrame
print("New Shape: ", general.shape)

### Handling Outliers

In [17]:
# à refaire

general_limit_value = {
    "Education" : (1,5),
    "EmployeeCount" : (0,1),
    "JobLevel" : (1,5)
}
employee_limit_value = {
    "EnvironmentSatisfaction" : (1,4),
    "JobSatisfaction" : (1,4),
    "WorkLifeBalance" : (1,4)
}
manager_limit_value = {
    "JobInvolvement" : (1,4),
    "PerformanceRating" : (1,4)
}

for column,(min_val,max_val) in general_limit_value.items() :
    general = general[(general[column] >= min_val) & (general[column] <= max_val)]

for column,(min_val,max_val) in employee_limit_value.items() :
    employeeSurvey = employeeSurvey[(employeeSurvey[column] >= min_val) & (employeeSurvey[column] <= max_val)]

for column,(min_val,max_val) in manager_limit_value.items() :
    managerSurvey = managerSurvey[(managerSurvey[column] >= min_val) & (managerSurvey[column] <= max_val)]


####
#rework
####



### Handling Missing Data

In [52]:
import missingno as msno

# Visualize missing data
#msno.matrix(housing_data)
#plt.show()

data_files = [general, employeeSurvey, managerSurvey, inTime, outTime]

# Impute missing values
# For numerical columns, use median imputation
for file in data_files:
    numerical_columns = file.select_dtypes(include=['float64', 'int64']).columns
    file[numerical_columns] = file[numerical_columns].fillna(file[numerical_columns].median())

    # For categorical columns, use mode imputation
    categorical_columns = file.select_dtypes(include=['object']).columns
    v = file[categorical_columns]
    va = v.mode()
    va
    #var = va.iloc[0]
    #file[categorical_columns] = file[categorical_columns].fillna(var)

    # Verify that there are no more missing values
    #print(file.isnull().sum())


### Export Data

In [15]:
import sys
import os

# Ajouter le chemin du projet au PATH
sys.path.append(os.path.abspath(".."))

from modules.data_import import exportDataCSV

exportDataCSV(general)